# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/anders/Documents/code/dataengineering/dataeng-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

For the first query we need to look up songplays by session_id and item_in_session number. We will need a table with these two fields as the partition key:

In [8]:
table1 = """
CREATE TABLE IF NOT EXISTS session_songs (
    session_id int,
    item_in_session int,
    artist text,
    song text,
    length decimal,
    PRIMARY KEY (session_id, item_in_session)
)
"""
session.execute(table1)

                    

Data is inserted into the table:

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

We can now output the artist of a particular song played in a given session:

In [10]:
query1 = """
SELECT artist, song, length
FROM session_songs
WHERE session_id = 338 AND item_in_session = 4
"""
rows = session.execute(query1)
pd.DataFrame(rows)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


#### Query 2

Here we need to look up songplays by user_id and session_id, so we will use these two fields as the partition key. We also would like to sort by item_in_session, so this field is included as a clustering column. The remaining columns include the fields we would like in the output:

In [11]:
table2 = """
CREATE TABLE IF NOT EXISTS songplays_by_user_session (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
)
"""
session.execute(table2)          

Data is inserted into the new table:

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_by_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

Now query2 can be executed, filtering by user_id and session_id, ordering by item_in_session, and outputting the relevant columns:

In [13]:
query2 = """
SELECT artist, song, first_name, last_name
FROM songplays_by_user_session
WHERE user_id = 10 AND session_id = 182
ORDER BY item_in_session
"""
rows = session.execute(query2)
pd.DataFrame(rows)

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Query 3

For this query we would like to look up users who listened to a certain song title. Since multiple users may have played the same song we need to include user_id in the primary key to make the primary key unique. For the output we want user first/last name, so we add these columns as well:

In [14]:
table3 = """
CREATE TABLE IF NOT EXISTS users_by_song (
    song text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song, user_id)
)
"""
session.execute(table3)
                    

Data is inserted here:

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

Now we may search for the users that listened to a particular song:

In [16]:
query3 = """
SELECT first_name, last_name
FROM users_by_song
WHERE song = 'All Hands Against His Own'
"""
rows = session.execute(query3)
pd.DataFrame(rows)

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE session_songs")
session.execute("DROP TABLE songplays_by_user_session")
session.execute("DROP TABLE users_by_song")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()